# GGIbot

This notebook controls execution of GGI using **pyautogui**.

It was written to automate downloading XML files for documents from Insects of Guam I and II
as part of a validation process.

This is a fragile hack which probably runs only on my particular setup.

In [1]:
import pyautogui as pag
import time
import mysecrets
import pandas as pd

In [2]:
def read_dataset_list():
    '''
    Reads ../dataset-list.md and returns a pandas dataframe
    '''
    
    # Read a markdown file, getting the header from the first row and inex from the second column
    # df = pd.read_table('../dataset-list.md', sep="|", header=0, index_col=1, skipinitialspace=True)
    df = pd.read_table('../dataset-list.md', sep="|", header=0, skipinitialspace=True)

    # Drop the left-most and right-most null columns 
    df = df.dropna(axis=1, how='all')

    # Drop the header underline row
    df = df.iloc[1:]  

    # Left-align strings and column headings
    # df = df.style.set_properties(**{'text-align': 'left'})
    # df = df.set_table_styles([dict(selector = 'th', props=[('text-align', 'left')])])

    # Strip whitespace from end of strings
    df = df.applymap(lambda x: x.strip() if isinstance(x, str) else x)

    # Strip whitespace from end of column headers
    df.columns = df.columns.str.strip()

    # Drop datasets with no title - we don't need to process these
    df = df.drop(df[df.title == 'no title'].index)
    return df

In [3]:
XMLDIR = '/home/aubrey/Desktop/data-mining-insects-of-guam/MatCit-Validator'

In [4]:
uuids = ['FF86FFC39D5F6775A307302F3260421D']

In [5]:
pag.PAUSE = 5

In [6]:
def open_GGI():
    pag.hotkey('ctrl','alt','t')
    pag.typewrite('cd GGI\n')
    pag.typewrite('java -jar GgImagineStarter.jar\n')
    pag.typewrite('\n')
    time.sleep(10)

    pag.click(1141, 565, duration=2) # select configuration
    pag.click(1469, 646, duration=2) # click configuration button
    pag.click(1590, 562, duration=2) # look and feel
    pag.click(1259, 635, duration=2) # select JAVA
    pag.click(1267, 709, duration=2) # OK
    pag.click(1143, 648, duration=2) # OK
    pag.click(1766, 171, duration=2) # maximize window

In [7]:
def login():
    '''
    This function spoofs a login by clicking on File | Load from GGI server items on the main GGI menu.
    The first time that a download is requested during a session results in a login dialog.
    
    My username is remembered by GGI, so this function provides only my password. Note that I keep 
    this private by reading it from mysecrets.py which is included in .gitignore. The pertinant line looks like
    "password = <my GGI password>".
    
    This function does not download a document from the GGI server, but exits after pressing the "cancel" button.    
    '''
    pag.click(  92,  69, duration=2) # File
    pag.click( 122, 134, duration=2) # load from GGI server
    pag.press('enter')
    pag.typewrite(mysecrets.password)  # password
    pag.press('enter')
    pag.click(1371, 594, duration=2) # cancel

In [8]:
def load_doc(uuid):
    pag.click(  92,  69, duration=2) # File
    pag.click( 122, 134, duration=2) # load from GGI server
    pag.typewrite(uuid)
    pag.press('enter')
    
# load_doc('FF86FFC39D5F6775A307302F3260421D')

In [9]:
def export_xml(uuid):
    pag.click(133,  72, duration=2) # Export
    pag.click(151, 154, duration=2) # Export XML
    xml_path = f'{XMLDIR}/{uuid}'
    pag.press('backspace')          # Remove space character in textbox
    pag.typewrite(xml_path)
    pag.press('enter')
    
# export_xml('FF86FFC39D5F6775A307302F3260421D')

In [10]:
def close_doc():
    pag.click(  92,  69, duration=2) # File
    pag.click( 132, 214, duration=2) # Close Document

#close_doc()

In [11]:
# MAIN

df = read_dataset_list()

# lets process the first 5 datasets
uuids = list(df.iloc[0:5].uuid.values)

open_GGI()
login()
for uuid in uuids:
    load_doc(uuid)
    export_xml(uuid)
    close_doc()
print('FINISHED')
print('Please update git repo for ~/Desktop/data-mining-insects-of-guam')

FINISHED
Please update git repo for ~/Desktop/data-mining-insects-of-guam
